# Geometry optimization

MyBigDFT comes with some classes implementing particular workflows of calculations. These workflows define a queue of jobs, that can easily be run sequentially, without having to worry about the Job context manager. They also generally define a particular post-processing procedure, run after all the BigDFT calculations in order to extract some meaningful imformation.

The example provided here shows how to obtain the geometry optimization of the N$_2$ molecule by using the [Geopt](https://mmoriniere.gitlab.io/MyBigDFT/geopt.html) workflow class.

## Initialization

You first need to import some useful classes to define a ground state calculation as well as the `Geopt` class, that is a workflow:

In [1]:
import numpy as np
from mybigdft import Posinp, Atom, InputParams, Job
from mybigdft.workflows import Geopt

## The [Geopt](https://mmoriniere.gitlab.io/MyBigDFT/geopt.html) class

This class allows to relax the input geometry of a given system in order to find the structure that minimizes the forces. It is meant to ease the creation of this type of calculation by automatically setting the main input parameters under the ``geopt`` key from the BigDFT input parameters.

Of course, you can specify the input parameters you want to use for all these calculations, via the ``inputparams`` argument. You may also want to give a specific name to the runs and a specific folder where to run them, as usual.

The base job has a given set of input parameters, and default values for the main parameters of the ``geopt`` key are automatically updated. The extra arguments of the ``geopt`` input parameters key can also be passed as keyword arguments.

In [2]:
atoms = [Atom('N', [0, 0, 0]), Atom('N', [0, 0, 1.0975])]  # experimental geometry
pos = Posinp(atoms, units="angstroem", boundary_conditions="free")
inp = InputParams()
base_job = Job(posinp=pos, inputparams=inp, name="N2", run_dir="N2/geopt/from_experimental_structure")
geopt = Geopt(base_job)

The workflow is run as usual:

In [3]:
geopt.run(nmpi=6, nomp=3)

/Users/maximemoriniere/Documents/Python/MyBigDFT/doc/source/notebooks/N2/geopt/from_experimental_structure
Logfile log-N2.yaml already exists!



Starting from the experimental geometry, the converged atomic structure found using default BigDFT parameters is the following:

In [4]:
print(geopt.final_posinp)

2   angstroem
free
N  -2.81097166026014e-21  -6.30580455586082e-22   0.00200502001148875
N   2.83565311682474e-21   7.23934260935048e-22   1.09549494660991



In [5]:
# Distance between both nitrogen atoms after geometry optimization
distance = geopt.final_posinp.distance(0, 1)
print(distance)

1.093489926598422


## Starting from another starting point

In [6]:
atoms_2 = [Atom('N', [0, 0, 0]), Atom('N', [0, 0, 1.05])]
pos_2 = Posinp(atoms_2, units="angstroem", boundary_conditions="free")
base_job_2 = Job(posinp=pos_2, inputparams=inp, name="N2", run_dir="N2/geopt/from_worse_staring_point")
geopt_2 = Geopt(base_job_2)

You can run this new workflow. Note that a warning was raised, telling us that, at some point, the input wavefunctions had to be recalculated: this is because the forces found for the initial geometry were high. The geometry optimization procedure then had to move the atoms by a large amount, which was sufficient for the input wavefunctions to be recalculated for the second geometry. For that reason, it is better to start from an initial position that is already close to a minimum. You might want to reach that "rather good" initial geometry with low quality input parameters (such as a small grid extension and a large grid step) before performing a more expensive one which gives converged results with respect to those input parameters.

In [7]:
geopt_2.run(nmpi=6, nomp=3)

/Users/maximemoriniere/Documents/Python/MyBigDFT/doc/source/notebooks/N2/geopt/from_worse_staring_point
Logfile log-N2.yaml already exists!



/Users/maximemoriniere/Documents/Python/MyBigDFT/mybigdft/iofiles/logfiles.py:265: UserWarning: The norm of the residue is too large, need to recalculate input wavefunctions
  warnings.warn(warning, UserWarning)


The same warning was actually found after the second step of the geometry optimization procedure. After that, the difference in initial geometry between two steps were low enough for the wavefunctions found at the end of the former step to be used as input guess for the next step, thus reducing the computational time. This is an indicator that your input geoemetry might not be close enough to the minimal geometry. The worst case scenario would be that the changes in the first steps of the geometry optimization procedure could leave you far away from the relaxed structure, and lead to a very long convergence (if any).

You can see readily see that the second geopt workflow did need more steps than the first one to reach convergence:

In [8]:
n_1 = len(geopt.queue[0].logfile)
n_2 = len(geopt_2.queue[0].logfile)
print(f"First geopt took {n_1} steps while second geopt took {n_2} steps.")

First geopt took 8 steps while second geopt took 27 steps.


It however reached a similar minimum:

In [9]:
distance = geopt_2.final_posinp.distance(0, 1)
print(distance)

1.093490323564331
